In [1]:
import tensorflow as tf
import numpy as np
import sys

from sklearn.metrics import mean_squared_error
from keras.callbacks import LearningRateScheduler
#from sklearn.preprocessing import MinMaxScaler ##########

sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
#sys.path.append('/Users/Think/Desktop/project')

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression

#Data handlers
from ann_framework.data_handlers.data_handler_Oscillator import OscillatorDataHandler, LinearPDEDataHandler

#Custom modules
from ann_framework import aux_functions

import aux_functions_stochastic
import analytic_functions
import loss_functions


Using TensorFlow backend.


In [2]:
#global variables

#deltas = [0.1, 0.1]
deltas = [0.1]

k = 1
c = 0.1
D = 1
num_fevals = 5

sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)

### Define tensorflow model

In [3]:
def create_placeholders(input_shape, output_shape):
    
    X = tf.placeholder(tf.float32, shape=(None,input_shape), name="X")
    y = tf.placeholder(tf.float32, shape=None, name="y")
    
    return X, y

def tf_model(X):
    
    l2_lambda_regularization = 0.20
    l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), 
                         name="fc1")
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    y = tf.layers.dense(A2, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="out")
    
    return y


def tf_ghazale(X):
    
    l2_lambda_regularization = 0.20
    #l1_lambda_regularization = 0.10
    
    A1 = tf.layers.dense(X, 10, activation=tf.nn.relu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False), 
                         kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_lambda_regularization), 
                         name="fc1")
    y = tf.layers.dense(A1, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        kernel_regularizer=tf.contrib.layers.l2_regularizer(l2_lambda_regularization), name="out")
    
    return y



def tf_model_yulin(X):
    
    A1 = tf.layers.dense(X, 32, activation=tf.nn.tanh, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),name="fc1")
    """
    dropout1 = tf.layers.dropout(inputs = A1, rate = 0.2)
    A2 = tf.layers.dense(A1, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc2")
    dropout2 = tf.layers.dropout(inputs = A2, rate = 0.2)
    A3 = tf.layers.dense(A2, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc3")
    dropout3 = tf.layers.dropout(inputs = A3, rate = 0.2)
    A4 = tf.layers.dense(A3, 20, activation=tf.nn.elu, 
                         kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                         kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1_lambda_regularization,l2_lambda_regularization), name="fc4")
    dropout4 = tf.layers.dropout(inputs = A4, rate = 0.2)
    """
    y = tf.layers.dense(A1, 1, activation=None, kernel_initializer=tf.contrib.layers.xavier_initializer(uniform=False),
                        name="out")
    #activation funcs
    
    return y


### Create Model TF

In [4]:
def tf_compiled_model(num_features, output_shape, num_fevals=1):
    tf.reset_default_graph()

    X, y = create_placeholders(num_features, output_shape)

    #y_pred = tf_model(X)
    #y_pred = tf_model_yulin(X)
    y_pred = tf_ghazale(X)
    
    #loss_function = loss_functions.squared_residual_function_wrapper2(k, c, D, deltas, num_fevals)
    loss_function =  loss_functions.linear_residual_function_wrapper(num_features, output_shape, deltas, num_fevals)
    cost, e = loss_function(X, y_pred, y)
    #reg_cost = tf.losses.get_regularization_loss()
    total_cost = e
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01, beta1=0.5).minimize(total_cost)

    return {'X_placeholder': X, 'y_placeholder': y, 'y_pred': y_pred, 'cost': cost, 'total_cost': total_cost, 'optimizer': optimizer}

### Create Tunable Model Tensorflow and assign data

In [5]:
"""

#scaler = MinMaxScaler()

dhandler_stochastic = OscillatorDataHandler()

#(data_scaler = scaler)

model = tf_compiled_model(2, 1, 5)

tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic)

tModel.load_data(verbose=1, cross_validation_ratio=0.2, x=[0, 0], boundaries=[10, 10], n=[50, 50])

#Real phi function
p_real_test, phi_real_test = analytic_functions.real_p(tModel.X_test[:,0], tModel.X_test[:,1], sigma_x, sigma_y)
p_real_train, phi_real_train = analytic_functions.real_p(tModel.X_train[:,0], tModel.X_train[:,1], sigma_x, sigma_y)
p_real_crossVal, phi_real_crossVal = analytic_functions.real_p(tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], sigma_x, sigma_y)

tModel.y_test = phi_real_test
tModel.y_train = phi_real_train
tModel.y_crossVal = phi_real_crossVal


tModel.print_data()
"""

"\n\n#scaler = MinMaxScaler()\n\ndhandler_stochastic = OscillatorDataHandler()\n\n#(data_scaler = scaler)\n\nmodel = tf_compiled_model(2, 1, 5)\n\ntModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', \n                                        data_handler=dhandler_stochastic)\n\ntModel.load_data(verbose=1, cross_validation_ratio=0.2, x=[0, 0], boundaries=[10, 10], n=[50, 50])\n\n#Real phi function\np_real_test, phi_real_test = analytic_functions.real_p(tModel.X_test[:,0], tModel.X_test[:,1], sigma_x, sigma_y)\np_real_train, phi_real_train = analytic_functions.real_p(tModel.X_train[:,0], tModel.X_train[:,1], sigma_x, sigma_y)\np_real_crossVal, phi_real_crossVal = analytic_functions.real_p(tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], sigma_x, sigma_y)\n\ntModel.y_test = phi_real_test\ntModel.y_train = phi_real_train\ntModel.y_crossVal = phi_real_crossVal\n\n\ntModel.print_data()\n"

## For linear PDE

In [6]:
#scaler = MinMaxScaler()

dhandler_stochastic = LinearPDEDataHandler()

#(data_scaler = scaler)

model = tf_compiled_model(1, 1, 2)

tModel = SequenceTunableModelRegression('ModelStochastic_SN_1', model, lib_type='tensorflow', 
                                        data_handler=dhandler_stochastic)

tModel.load_data(verbose=1, cross_validation_ratio=0.2, x=0, boundaries=[0, 10], n=100)
#Real phi function
tModel.y_test = analytic_functions.real_y(tModel.X_test[:,0])
tModel.y_train = analytic_functions.real_y(tModel.X_train[:,0])
tModel.y_crossVal = analytic_functions.real_y(tModel.X_crossVal[:,0])

tModel.print_data()

Loading data for the first time
Reloading data due to parameter change
Loading data. Cros-Validation ratio 0.2
[[ 0.        ]
 [ 0.05025126]
 [ 0.10050251]
 [ 0.15075377]
 [ 0.20100503]
 [ 0.25125628]
 [ 0.30150754]
 [ 0.35175879]
 [ 0.40201005]
 [ 0.45226131]
 [ 0.50251256]
 [ 0.55276382]
 [ 0.60301508]
 [ 0.65326633]
 [ 0.70351759]
 [ 0.75376884]
 [ 0.8040201 ]
 [ 0.85427136]
 [ 0.90452261]
 [ 0.95477387]
 [ 1.00502513]
 [ 1.05527638]
 [ 1.10552764]
 [ 1.15577889]
 [ 1.20603015]
 [ 1.25628141]
 [ 1.30653266]
 [ 1.35678392]
 [ 1.40703518]
 [ 1.45728643]
 [ 1.50753769]
 [ 1.55778894]
 [ 1.6080402 ]
 [ 1.65829146]
 [ 1.70854271]
 [ 1.75879397]
 [ 1.80904523]
 [ 1.85929648]
 [ 1.90954774]
 [ 1.95979899]
 [ 2.01005025]
 [ 2.06030151]
 [ 2.11055276]
 [ 2.16080402]
 [ 2.21105528]
 [ 2.26130653]
 [ 2.31155779]
 [ 2.36180905]
 [ 2.4120603 ]
 [ 2.46231156]
 [ 2.51256281]
 [ 2.56281407]
 [ 2.61306533]
 [ 2.66331658]
 [ 2.71356784]
 [ 2.7638191 ]
 [ 2.81407035]
 [ 2.86432161]
 [ 2.91457286]
 [ 2

/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Train model Tensorflow

In [7]:
tModel.epochs = 200
#lrate = LearningRateScheduler(aux_functions.step_decay)
minibatches_function_handle = aux_functions_stochastic.get_minibatches_linear

#tf.get_variable("deltas", initializer=deltas)

sess = tf.Session()

#writer = tf.summary.FileWriter('./graphs', sess.graph)

tModel.train_model(tf_session=sess, get_minibatches_function_handle=minibatches_function_handle, 
                   verbose=1, deltas=deltas)

    # cost_reg: total_cost -> e
    # cost: R

[[ 6.7839196 ]
 [ 4.87437186]
 [ 0.15075377]
 [ 1.75879397]
 [ 5.67839196]
 [ 5.27638191]
 [ 2.11055276]
 [ 5.92964824]
 [10.        ]
 [ 5.1758794 ]
 [ 3.71859296]
 [ 4.27135678]
 [ 3.2160804 ]
 [ 6.98492462]
 [ 5.57788945]
 [ 3.46733668]
 [ 6.83417085]
 [ 1.15577889]
 [ 7.88944724]
 [ 2.51256281]
 [ 1.10552764]
 [ 6.53266332]
 [ 4.72361809]
 [ 8.59296482]
 [ 5.22613065]
 [ 3.11557789]
 [ 2.56281407]
 [ 6.48241206]
 [ 5.52763819]
 [ 4.07035176]
 [ 8.34170854]
 [ 1.25628141]
 [ 8.14070352]
 [ 6.03015075]
 [ 5.12562814]
 [ 3.86934673]
 [ 6.58291457]
 [ 1.50753769]
 [ 5.32663317]
 [ 0.25125628]
 [ 2.36180905]
 [ 6.08040201]
 [ 3.81909548]
 [ 0.        ]
 [ 2.7638191 ]
 [ 7.38693467]
 [ 4.62311558]
 [ 7.78894472]
 [ 8.49246231]
 [ 4.0201005 ]
 [ 0.45226131]
 [ 8.64321608]
 [ 4.3718593 ]
 [ 7.18592965]
 [ 4.47236181]
 [ 4.67336683]
 [ 1.00502513]
 [ 5.42713568]
 [ 8.04020101]
 [ 9.54773869]
 [ 9.34673367]
 [ 0.60301508]
 [ 5.47738693]
 [ 2.96482412]
 [ 9.74874372]
 [ 7.83919598]
 [ 2.91457

InvalidArgumentError: Incompatible shapes: [432,1] vs. [216,1]
	 [[node gradients/Sub_1_grad/BroadcastGradientArgs (defined at <ipython-input-4-29e4eb245c30>:16)  = BroadcastGradientArgs[T=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](gradients/Sub_1_grad/Shape, gradients/Sub_1_grad/Shape_1)]]

Caused by op 'gradients/Sub_1_grad/BroadcastGradientArgs', defined at:
  File "/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/david

In [ ]:
display_points = 50

"""
#Evaluate real model
X_test = tModel.X_crossVal

#Real phi function
sigma_x = np.sqrt(D / (k * c))
sigma_y = np.sqrt(D / c)
p_real, phi_real = analytic_functions.real_p(X_test[:,0], X_test[:,1], sigma_x, sigma_y)

tModel.y_crossVal = phi_real

#p_real = np.ravel(p_real)
#phi_real = np.ravel(phi_real)

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
#phi_pred = np.ravel(tModel.y_predicted)
phi_pred = tModel.y_predicted

d = 2 * np.pi * sigma_x * sigma_y
c_not = 1/d
p_pred = c_not * np.exp(-phi_pred)
"""

tModel.evaluate_model(['mse', 'rmse'], cross_validation=True, tf_session=sess)
X_test = tModel.X_crossVal
phi_pred = tModel.y_predicted
phi_real = tModel.y_crossVal
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
mse = cScores['mse']
time = tModel.train_time

total_points = len(phi_pred)
sample_array = list(range(total_points))

sample_points = np.random.choice(sample_array, display_points)

phi_real_sampled = phi_real[sample_points]
phi_pred_sampled = phi_pred[sample_points]
X_sampled = X_test[sample_points,:]

i = range(len(phi_pred_sampled))


for i, phi_real_display, phi_pred_display in zip(i, phi_real, phi_pred):
    print('xy {}, Real Phi {}, Predicted Phi {}'.format(X_test[i], phi_real_display, phi_pred_display))

#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("MSE: {}".format(mse))
print("Time : {} seconds".format(time))

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig1 = plt.figure(1)
ax1 = Axes3D(fig1)
pred_phi_plot = Axes3D.scatter(ax1, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], phi_pred)
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('phi(x, y)_pred')

plt.show()

In [ ]:
fig2 = plt.figure(2)
ax2 = Axes3D(fig2)
real_phi_plot = Axes3D.scatter(ax2, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], phi_real)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('phi(x, y)_real')

plt.show()

In [ ]:
fig3 = plt.figure(3)
ax3 = Axes3D(fig3)
pred_p_plot = Axes3D.scatter(ax3, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], p_pred)
ax3.set_xlabel('x')
ax3.set_ylabel('y')
ax3.set_zlabel('p(x, y)_pred')

plt.show()

In [ ]:
fig4 = plt.figure(4)
ax4 = Axes3D(fig4)
real_p_plot = Axes3D.scatter(ax4, tModel.X_crossVal[:,0], tModel.X_crossVal[:,1], p_real)
ax4.set_xlabel('x')
ax4.set_ylabel('y')
ax4.set_zlabel('p(x, y)_real')

plt.show()

In [ ]:
def real_derivatives(X, sigma_x, sigma_y):
    x1 = X[:, 0]
    x2 = X[:, 1]
    
    first_order_dx = x1/(sigma_x**2)
    first_order_dy = x2/(sigma_y**2)
    second_order_dy = 1/(sigma_y**2)
    
    return first_order_dx, first_order_dy, second_order_dy

dx, dy, ddy = real_derivatives(X_test, sigma_x, sigma_y)

In [ ]:
dx, dy, ddy = analytic_functions.real_derivatives(tModel.X_crossVal, sigma_x, sigma_y)

#for i in range(len(dx)):
 #   print('dx {}, dy {}, ddy {}'.format(dx[i], dy[i], ddy[i]))